In [ ]:
import pandas as pd

# Load the data, correcting the file path
df = pd.read_csv('data/IMDb Movies India.csv')

# Display basic info
print(df.head())
print(df.info())

FileNotFoundError: [Errno 2] No such file or directory: 'IMDb Movies India.csv'

In [1]:
import numpy as np

# Create a copy of the dataframe
df_cleaned = df.copy()

# Drop rows where 'Rating' is null
df_cleaned.dropna(subset=['Rating'], inplace=True)

# Clean and convert 'Year' column
df_cleaned['Year'] = df_cleaned['Year'].str.replace(r'[()]', '', regex=True).astype(int)

# Clean and convert 'Duration' column
df_cleaned['Duration'] = df_cleaned['Duration'].str.replace(' min', '', regex=False)
df_cleaned['Duration'] = pd.to_numeric(df_cleaned['Duration'], errors='coerce')
df_cleaned['Duration'].fillna(df_cleaned['Duration'].median(), inplace=True)

# Clean and convert 'Votes' column
df_cleaned['Votes'] = df_cleaned['Votes'].str.replace(',', '', regex=False).astype(int)

# Handle missing values in categorical columns
df_cleaned['Genre'].fillna('Unknown', inplace=True)
df_cleaned['Director'].fillna('Unknown', inplace=True)
df_cleaned['Actor 1'].fillna('Unknown', inplace=True)
df_cleaned['Actor 2'].fillna('Unknown', inplace=True)
df_cleaned['Actor 3'].fillna('Unknown', inplace=True)

NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Handle the 'Genre' column by splitting and one-hot encoding
genres = df_cleaned['Genre'].str.split(', ')
mlb = MultiLabelBinarizer()
genre_matrix = mlb.fit_transform(genres)
genre_df = pd.DataFrame(genre_matrix, columns=mlb.classes_, index=df_cleaned.index)
df_processed = pd.concat([df_cleaned, genre_df], axis=1)

# Drop original 'Genre' and 'Name'
df_processed.drop(columns=['Name', 'Genre'], inplace=True)

# Function to encode top N features
def get_top_n_encoded_features(df, column, n=150):
    top_n_values = df[column].value_counts().nlargest(n).index.tolist()
    df[f'is_top_{column}'] = df[column].apply(lambda x: x if x in top_n_values else 'Other')
    return pd.get_dummies(df, columns=[f'is_top_{column}'], prefix=column)

# Apply encoding to Director and Actor columns
df_processed = get_top_n_encoded_features(df_processed, 'Director')
df_processed = get_top_n_encoded_features(df_processed, 'Actor 1')
df_processed = get_top_n_encoded_features(df_processed, 'Actor 2')
df_processed = get_top_n_encoded_features(df_processed, 'Actor 3')

# Drop the original high-cardinality columns
df_processed.drop(columns=['Director', 'Actor 1', 'Actor 2', 'Actor 3'], inplace=True)